In [9]:
import pandas as pd
import os
import json

os.getcwd()

In [5]:
file_name = "subsampleprint(data)d_drug_info.json"
# Open and read the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [108]:
drugs = df.columns
rows = df.index
disease_prompts = dict()


for drug in drugs:
    diseases = df[drug]['Disease Information']
    receptors = df[drug]['Additional Information']['targets']
    rec_data = dict()
    k = list(df[drug]['Active Ingredients'].keys())
    smile = df[drug]['Active Ingredients'][k[0]]
        
    for r in receptors:
        rec_data[r['Label']] = {'Amino acid sequence' : r['Detailed Info']['Amino acid sequence'], 
                                'General Function': r['Detailed Info']['General Function'],
                                'Specific Function' : r['Detailed Info']['Specific Function']}                   
    
    for dis in diseases:
        if diseases[dis]==None:
            info =''
        else:
            info = ', is '+ diseases[dis]
        disease_prompts[dis] = {'info': info, 'receptors': rec_data, 'drug_smile' : smile}
        

In [110]:
templates_prompts_w_smiles = dict()
for d in disease_prompts:
    receptors = []
    for r in disease_prompts[d]['receptors']:
        gf=disease_prompts[d]['receptors'][r]['General Function']
        sf = disease_prompts[d]['receptors'][r]['Specific Function']
        ac = disease_prompts[d]['receptors'][r]['Amino acid sequence']
        receptor = f'{r}, is {gf}, and {sf}. Its amino acid sequence is {ac}'
        receptors.append(receptor)
    allreceptors = '.'.join(receptors)
    i=disease_prompts[d]['info']
    prompt = f'You are an expert in drug discovery, please generate a novel binder molecule for the following disease, Respond with a SMILES of a possible binder molecule. The disease is {d} {i}. It has the following receptors: {allreceptors}'
    templates_prompts_w_smiles[d] = {'prompt':prompt, 'smile_output': disease_prompts[d]['drug_smile']}
            

In [111]:
#Example
templates_prompts_w_smiles['Muscle Spasms']['prompt']

'You are an expert in drug discovery, please generate a novel binder molecule for the following disease, Respond with a SMILES of a possible binder molecule. The disease is Muscle Spasms . It has the following receptors: Prostaglandin G/H synthase 2, is Prostaglandin-endoperoxide synthase activity, and Converts arachidonate to prostaglandin H2 (PGH2), a committed step in prostanoid synthesis. Constitutively expressed in some tissues in physiological conditions, such as the endothelium, kidney and brain, and in pathological conditions, such as in cancer. PTGS2 is responsible for production of inflammatory prostaglandins. Up-regulation of PTGS2 is also associated with increased cell adhesion, phenotypic changes, resistance to apoptosis and tumor angiogenesis. In cancer cells, PTGS2 is a key step in the production of prostaglandin E2 (PGE2), which plays important roles in modulating motility, proliferation and resistance to apoptosis.. Its amino acid sequence is >lcl|BSEQ0021832|Prostagla